<a href="https://colab.research.google.com/github/CabralMM/BigDataCourse/blob/main/Prophet_Predecir_el_valor_de_B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prophet - Predecir el valor de B3


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [ ]:
# Instalamos el modelo predictivo Prophet
!python -m pip install prophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [ ]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [ ]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-06-07'
start_date = '2016-01-01'
# Descargar el dataframe
b3_df = yf.download('B3SA3.SA',start_date, today).reset_index()

b3_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
1844,2023-05-31,$13.37,$13.58,$13.13,$13.32,$13.32,70207400
1845,2023-06-01,$13.45,$14.42,$13.38,$14.30,$14.30,62415600
1846,2023-06-02,$14.50,$14.69,$14.26,$14.42,$14.42,44298200
1847,2023-06-05,$14.46,$14.56,$14.16,$14.46,$14.46,38760200
1848,2023-06-06,$14.51,$14.70,$14.26,$14.41,$14.41,57442500


In [ ]:
# Validamos que no hay datos vacíos
b3_df.empty

False

In [ ]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = b3_df[['Date','Open']]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df.rename(columns = new_names, inplace = True)
df

,ds,y
0,2016-01-04,$3.58
1,2016-01-05,$3.50
2,2016-01-06,$3.59
3,2016-01-07,$3.58
4,2016-01-08,$3.50
...,...,...
1844,2023-05-31,$13.37
1845,2023-06-01,$13.45
1846,2023-06-02,$14.50
1847,2023-06-05,$14.46


In [ ]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [ ]:
df.tail()


,ds,y
1844,2023-05-31,$13.37
1845,2023-06-01,$13.45
1846,2023-06-02,$14.50
1847,2023-06-05,$14.46
1848,2023-06-06,$14.51


In [ ]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = "B3 - Brasil"
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [ ]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet(seasonality_mode='multiplicative')


In [ ]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpm4hh3j51/wl8ffk7h.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpm4hh3j51/2_9dcmap.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=98247', 'data', 'file=/tmp/tmpm4hh3j51/wl8ffk7h.json', 'init=/tmp/tmpm4hh3j51/2_9dcmap.json', 'output', 'file=/tmp/tmpm4hh3j51/prophet_modelcpgreb6a/prophet_model-20230607083543.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
08:35:43 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
08:35:46 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [ ]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
2209,2024-06-01
2210,2024-06-02
2211,2024-06-03
2212,2024-06-04
2213,2024-06-05


In [ ]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future = future[ future['ds'].dt.dayofweek < 5 ]

<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [ ]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future) 

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2105,2024-05-30,$11.29,$-1.52,$23.83
2106,2024-05-31,$11.33,$-1.80,$23.84
2107,2024-06-03,$11.30,$-2.02,$23.96
2108,2024-06-04,$11.26,$-1.97,$24.06
2109,2024-06-05,$11.22,$-1.89,$24.01


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [ ]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)